TRABALHO PRÁTICO

     Loja LIDER Eletrodomésticos
    
Esse é o meu trabalho ele simula um script de uma loja de eletrodomésticos.Onde possui as seguintes funcionalidade:

O usuário consegue se cadastrar e fazer login, no caso se o usuário for funcionário da loja terá direito a modificar a sua categoria.
Quando o usuário comum faz o seu cadastro automaticamente é cadastrado na categoria "cliente", se for funciónario na empresa tem a 
opção de colocar a sua categoria que será referente ao seu cargo de ocupação na empresa.
O funcionário na categoria "gerente" é o único a ter a permissão de editar os produtos, podendo adicionar ou excluir produtos.

Estou entregando o trabalho com atraso pois tive dificuldades em entender o mecanismo de funcionamento para leitura e edição de dados.
Agradeço a compreensão.

In [23]:
#Funções para funcionamento do sistema

import csv
import pandas as pd
import unicodedata
import unidecode
produtos_arquivo = 'produtos_loja_lider.csv' # variável que recebe o nome do arquivo com os produtos da loja
usuarios_arquivo = 'usuarios_loja_lider.csv' # variável que recebe o nome do arquivo com os usuários cadastrados da loja
nome_usuario = ' '  # variável que recebe o nome do usuário para cadastro ou login
senha = ' '         # variável que recebe a senha do usuário para cadastro ou login
categoria = ' '     # variável que recebe a categoria do usuário para cadastro ou login
status_categoria = ' ' # variável que recebe e verifica se a categoria do usuário tem permissão de edição

def pesquisar_produtos():                    # função que permite que o usuário pesquise o produdo pelo nome
    pdt = input('Digite o nome do produto:') # regularizando de forma que pode receber um nome semelhante e será reconhecido
    pdt = unidecode.unidecode(pdt).casefold()
    produtos_lider = pd.read_csv(produtos_arquivo, encoding= 'UTF8', sep= ',')  
    for i in produtos_lider.itertuples():   
        produtos_lider_caractere = i[2]
        normalizar = unicodedata.normalize('NFD', produtos_lider_caractere)
        normalizar = normalizar.encode('ascii', 'ignore').decode('utf8').casefold()
        if pdt in normalizar:
            print(' ')
            print(i[2], '   ', i[3])  
                                # função que permite que o usuário pesquise o produdo pela categoria
def pesquisar_categoria():      # regularizando de forma que pode receber um nome semelhante e será reconhecido
    ctg = input('Digite a categoria do produto (ex:Móveis, Eletrodomésticos, Eletroportáteis,...):')
    ctg = unidecode.unidecode(ctg).casefold()
    produtos_lider = pd.read_csv(produtos_arquivo, encoding= 'UTF8', sep= ',')
    for i in produtos_lider.itertuples():   
        produtos_lider_caractere = i[4] 
        normalizar = unicodedata.normalize('NFD', produtos_lider_caractere)
        normalizar = normalizar.encode('ascii', 'ignore').decode('utf8').casefold()
        if ctg in normalizar:
            print(' ')
            print(i[2],'   ', i[3])
       
def cadastrar_usuario():           # função para cadastrar usuário e definir o método de cadastro
    if verifica_usuario():
        print('Usuário já existe! Cadastro não realizado')
    else:
        print('Você é funcionário nessa empresa?\n1-Sim\n2-Não')
        opcao = int(input())
        if opcao == 1:
            codigo_funcionario = input('Digite o codigo da empresa:')
            if codigo_funcionario == '369': 
                senha = input('Digite uma senha:')
                categoria = input('Digite sua categoria de cargo(gerente,funcionário ou vendendor):')
                with open(usuarios_arquivo, 'a+', encoding='utf8') as usuario:
                    writer = csv.writer(usuario)
                    writer.writerow((nome_usuario,senha,categoria))
                    print('Cadastro realizado com sucesso!')
            else:
                print('Código incorreto!')
        elif opcao == 2:
            senha = input('Digite uma senha:')
            categoria = 'cliente'
            with open(usuarios_arquivo, 'a+', encoding='utf8') as usuario:
                writer = csv.writer(usuario)
                writer.writerow((nome_usuario,senha,categoria))
                print('Cadastro realizado com sucesso!')

def verifica_usuario():          # função que verifica se o usuário já existe no sistema não permitindo dois cadastros com o mesmo usuário
    with open(usuarios_arquivo, 'r') as usuario:
        texto = csv.reader(usuario)
        for i in texto:
            if nome_usuario in i:
                return True    
      


def limpar_arquivo():           # função que limpa o arquivo para atualizar no caso de edição de um produto
    with open(produtos_arquivo, 'w', encoding='utf8') as produtos_lider1:
        limpar_arquivo = csv.writer(produtos_lider1)
        
def fazer_login():              # função que determina o modo de acesso do usuário ao sistema 
    if verifica_login():        # reconhecendo se o usuário for gerente lhe dando acesso de edição no sistema
        print('Login realizado com sucesso!')
        with open(usuarios_arquivo, 'r') as usuario:
            texto = csv.reader(usuario)
            for i in texto:
                if nome_usuario in i and senha in i:
                    status_categoria = i[2]
        if status_categoria == 'gerente':
            print('Você é gerente nesta loja deseja editar algum produto?')
            opcao_editar = int(input('Digite 1-Editar ou 2-Somente acessar a loja.'))
            while opcao_editar != 3:
                produtos_lider = pd.read_csv(produtos_arquivo, usecols=['Produto', 'Preço', 'Categoria'])  
                display(produtos_lider)
                opcao_editar = int(input('Digite 1 para excluir um produto, 2 para adicionar um produto ou 3 para sair:'))
                if opcao_editar == 1:
                    excluir_linha = int(input('Digite o número da linha que deseja excluir:'))
                    excluir_produto = pd.read_csv(produtos_arquivo, usecols=['Produto', 'Preço', 'Categoria'], encoding= 'UTF8', sep= ',')
                    excluir_produto = excluir_produto.drop(excluir_linha)
                    limpar_arquivo()
                    excluir_produto.to_csv(produtos_arquivo)
                    print(f'Linha {excluir_linha} excluida com sucesso!') 
                elif opcao_editar == 2:
                    produto = input('Digite o nome do produto:')
                    preco = input('Digite o preço do produto(formato 000,00):')
                    categoria_produto = input('Digite a categoria do produto:')
                    with open(produtos_arquivo, 'a+', encoding='utf8') as produtos_cadastro:
                        writer = csv.writer(produtos_cadastro)
                        writer.writerow((' ', produto,'R$ '+ preco,categoria_produto)) 
                        produtos_lider = pd.read_csv(produtos_arquivo, usecols=['Produto', 'Preço', 'Categoria'])  
                        display(produtos_lider)
                if opcao_editar == 3:
                    break        
        acessar_loja()
    else:
        print('Usuário ou senha incorreto')

def verifica_login():   # função que verifica se usuário e senha são correspondentes permitindo o acesso
    with open(usuarios_arquivo, 'r') as usuario:
        texto = csv.reader(usuario)
        for i in texto:
            if nome_usuario in i and senha in i:
                return True    

def acessar_loja():   # função que dá acesso à loja permitindo a visualização dos produtos
    print('Deseja visualizar todos os produtos da loja ou prefere pesquisar por produto ou categoria?')
    opcao_pesquisa = int(input('Digite 1-para todos 2-para produto e 3-para categoria:'))
    if opcao_pesquisa == 1:
        produtos_lider = pd.read_csv(produtos_arquivo, encoding= 'UTF8', sep= ',')
        for i in produtos_lider.itertuples():
            print(' ')
            print(i[2] ,'   ', i[3])
    elif opcao_pesquisa == 2:
        pesquisar_produtos()
    elif opcao_pesquisa == 3:
        pesquisar_categoria()
        
       


In [24]:

print('Bem vindo a LIDER Loja de Eletrodomésticos')
print(' ')
while True:
    print('1-Cadastrar\n2-Login\n3-Sair')
    op = int(input('Escolha uma opção'))
    print(op)

    if op ==1:
        nome_usuario = input('Digite seu novo usuário:') 
        cadastrar_usuario()
    elif op == 2:
        nome_usuario = input('Digite seu nome de usuário:')
        senha = input('Digite a sua senha:')
        fazer_login()
    elif op == 3:
        break
    else:
        print('Opção inválida')
        print('-' * 15)

Bem vindo a LIDER Loja de Eletrodomésticos
 
1-Cadastrar
2-Login
3-Sair


Escolha uma opção 1


1


Digite seu novo usuário: marta


Você é funcionário nessa empresa?
1-Sim
2-Não


...

Deseja visualizar todos os produtos da loja ou prefere pesquisar por produto ou categoria?


Digite 1-para todos 2-para produto e 3-para categoria: 1


 
Armário de cozinha     R$ 450,00
 
Televisão 43 polegadas     R$ 1650,00
 
Televisão 50 polegadas     R$ 3200,00
 
Televisão 55 polegadas     R$ 3990,00
 
Mesa de madeira com 6 cadeiras     R$ 850,00
 
Geladeira duplex     R$ 3500,00
 
Televisão 60 polegadas     R$ 4500,00
 
Guarda-roupas 6 portas     R$ 750,00
 
Guarda-roupas 8 portas     R$ 850,00
 
Mesa com 4 cadeiras     R$ 600,00
 
Cama box Casal     R$ 1500,00
 
Cama Box Solteiro     R$ 1100,00
 
Rack para Televisão até 50 polegadas     R$ 650,00
 
Sofá 2 e 3 lugares     R$ 1250,00
 
Notebook     R$ 2600,00
 
Celular 5G     R$ 1800,00
 
Liquidificador     R$ 180,00
1-Cadastrar
2-Login
3-Sair


Escolha uma opção 3


3


In [25]:
with open('usuarios_loja_lider.csv', 'r') as usuario:
        texto = csv.reader(usuario)
        for i in texto:
            print(i)
    
        

['Nome de Usuário', 'Senha', 'Categoria']
[]
['erick', '12345', 'gerente']
[]
['bob', '12366', 'gerente']
[]
['regis', '12377', 'gerente']
[]
['maria', '45611', 'funcionario']
[]
['joao', '45622', 'funcionario']
[]
['jose', '45688', 'funcionario']
[]
['joaquim', '45698', 'funcionario']
[]
['raul', '78944', 'cliente']
[]
['kaique', '78915', 'cliente']
[]
['kaka', '78952', 'cliente']
[]
['joca', '78918', 'cliente']
[]
['natan', '89564', 'cliente']
[]
['henrique', '67855', 'vendedor']
[]
['heraldo', '45678', 'funcionÃ¡rio']
[]
['davi', '45688', 'cliente']
[]
['barbosa', '45690', 'cliente']
[]
['marta', '98552', 'cliente']
[]


In [26]:
import pandas as pd

produtos_loja = pd.read_csv("produtos_loja_lider.csv", encoding= 'UTF8', sep= ',')
produtos_loja

,Unnamed: 0,Produto,Preço,Categoria
0,0,Armário de cozinha,"R$ 450,00",Móveis
1,1,Televisão 43 polegadas,"R$ 1650,00",Eletrodomésticos
2,2,Televisão 50 polegadas,"R$ 3200,00",Eletrodomésticos
3,3,Televisão 55 polegadas,"R$ 3990,00",Eletrodomésticos
4,4,Mesa de madeira com 6 cadeiras,"R$ 850,00",Móveis
5,5,Geladeira duplex,"R$ 3500,00",Eletrodomésticos
6,6,Televisão 60 polegadas,"R$ 4500,00",Eletrodomésticos
7,7,Guarda-roupas 6 portas,"R$ 750,00",Móveis
8,8,Guarda-roupas 8 portas,"R$ 850,00",Móveis
9,9,Mesa com 4 cadeiras,"R$ 600,00",Móveis
